# 0. Import

In [ ]:
!pip install optuna

In [ ]:
pip install -U ipywidgets

In [ ]:
import pandas as pd
import numpy as np

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import  OrdinalEncoder, StandardScaler, MinMaxScaler, RobustScaler, OneHotEncoder, PolynomialFeatures
from sklearn.model_selection import train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.decomposition import PCA
from sklearn.ensemble import StackingClassifier
import optuna
# from xgboost import XGBClassifier
# from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier, Pool

from sklearn.utils.class_weight import compute_class_weight


import seaborn as sns 
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 1. Data Load

In [ ]:
train = pd.read_csv('./train.csv').drop(columns=['ID'])
test = pd.read_csv('./test.csv').drop(columns=['ID'])

X_train = train.drop('임신 성공 여부', axis=1)
y = train['임신 성공 여부']

In [ ]:
numeric_columns = [
    "임신 시도 또는 마지막 임신 경과 연수",
    "총 생성 배아 수",  #used ####
    "미세주입된 난자 수",
    "미세주입에서 생성된 배아 수",
    "이식된 배아 수",   #used
    "미세주입 배아 이식 수",  #####
    "저장된 배아 수",
    "미세주입 후 저장된 배아 수",
    "해동된 배아 수",
    "해동 난자 수", #used
    "수집된 신선 난자 수",  #used #####
    "저장된 신선 난자 수",
    "혼합된 난자 수",   #used #####
    "파트너 정자와 혼합된 난자 수", #####
    "기증자 정자와 혼합된 난자 수",
    "난자 채취 경과일", #####
    "난자 해동 경과일",
    "난자 혼합 경과일",
    "배아 이식 경과일", #####
    "배아 해동 경과일"
]
print(len(numeric_columns))

In [ ]:
categorical_columns = [
    "시술 시기 코드",
    "시술 당시 나이", #used # most important factor
    "시술 유형",
    "특정 시술 유형",
    "배란 자극 여부",   #used -> eliminated # 없앴더니 성능 감소
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",    #used
    "남성 부 불임 원인",    #used
    "여성 주 불임 원인",    #used
    "여성 부 불임 원인",    #used
    "부부 주 불임 원인",    #used
    "부부 부 불임 원인",    #used
    "불명확 불임 원인", #used # 없 감
    "불임 원인 - 난관 질환",    #used
    "불임 원인 - 남성 요인",    #used
    "불임 원인 - 배란 장애",    #used
    "불임 원인 - 여성 요인",    #used -> eliminated   # 얘는 그대로
    "불임 원인 - 자궁경부 문제",    #used
    "불임 원인 - 자궁내막증",   #used
    "불임 원인 - 정자 농도",    #used
    "불임 원인 - 정자 면역학적 요인",   #used -> eliminated # 없앴더니 성능 감소
    "불임 원인 - 정자 운동성",  #used
    "불임 원인 - 정자 형태",    #used
    "배아 생성 주요 이유",
    "총 시술 횟수", #used
    "클리닉 내 총 시술 횟수",
    "IVF 시술 횟수",
    "DI 시술 횟수",
    "총 임신 횟수", #used
    "IVF 임신 횟수",
    "DI 임신 횟수",
    "총 출산 횟수", #used
    "IVF 출산 횟수",
    "DI 출산 횟수",
    "난자 출처",    #used -> eliminated
    "정자 출처",    #used -> eliminated
    "난자 기증자 나이",
    "정자 기증자 나이",
    "동결 배아 사용 여부",  #used
    "신선 배아 사용 여부",  #used
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부"
]

In [ ]:

Categorical_non_ordinal=[
    "시술 시기 코드",
    "시술 유형",
    "배란 자극 여부",
    "배란 유도 유형",
    "단일 배아 이식 여부",
    "착상 전 유전 검사 사용 여부",
    "착상 전 유전 진단 사용 여부",
    "남성 주 불임 원인",
    "남성 부 불임 원인",
    "여성 주 불임 원인",
    "여성 부 불임 원인",
    "부부 주 불임 원인",
    "부부 부 불임 원인",
    "불명확 불임 원인",
    "불임 원인 - 난관 질환",
    "불임 원인 - 남성 요인",
    "불임 원인 - 배란 장애",
    "불임 원인 - 여성 요인",
    "불임 원인 - 자궁경부 문제",
    "불임 원인 - 자궁내막증",
    "불임 원인 - 정자 농도",
    "불임 원인 - 정자 면역학적 요인",
    "불임 원인 - 정자 운동성",
    "불임 원인 - 정자 형태",
    "배아 생성 주요 이유",
    "난자 출처",
    "정자 출처",
    "동결 배아 사용 여부",
    "신선 배아 사용 여부",
    "기증 배아 사용 여부",
    "대리모 여부",
    "PGD 시술 여부",
    "PGS 시술 여부",
    "특정 시술 유형"
] #34

# 2. PreProcessing0

# 3. Basic PreProcessing

### 3-1. Data Drop & Impute

In [ ]:
out_list = ['시술 유형', '착상 전 유전 진단 사용 여부', '남성 주 불임 원인', '남성 부 불임 원인', '여성 주 불임 원인', 
              '여성 부 불임 원인', '부부 주 불임 원인', '부부 부 불임 원인', '불명확 불임 원인', '불임 원인 - 여성 요인', 
              '불임 원인 - 자궁경부 문제', '불임 원인 - 자궁내막증', '불임 원인 - 정자 농도', 
              '불임 원인 - 정자 운동성', 'DI 임신 횟수', 'DI 출산 횟수', '동결 배아 사용 여부', 
              '신선 배아 사용 여부', '기증 배아 사용 여부', '대리모 여부']

numeric_columns = list(set(numeric_columns) - set(out_list))
categorical_columns = list(set(categorical_columns) - set(out_list))
Categorical_non_ordinal = list(set(Categorical_non_ordinal) - set(out_list))

In [ ]:
X_train = X_train.drop(columns=out_list)
test = test.drop(columns=out_list)

##### -Numeric

In [ ]:
''' Numeric Drop '''
#numeric - 결측치 80% 이상 drop
for col in numeric_columns:
    print(X_train[col].isnull().sum())
numeric_null_columns = [ col for col in numeric_columns if X_train[col].isnull().sum() != 0 ]

numeric_null_ratio = X_train[numeric_null_columns].isnull().mean().sort_values(ascending=False)
print(numeric_null_ratio)

drop_columns = numeric_null_ratio[numeric_null_ratio >= 0.8].index
drop_columns = drop_columns.drop('임신 시도 또는 마지막 임신 경과 연수')
X_train = X_train.drop(columns=drop_columns)
test = test.drop(columns=drop_columns)

numeric_columns = list(set(numeric_columns) - set(drop_columns))
print("📌"+drop_columns)


In [ ]:
''' Numeric Impute 2 : KNN Imputer for model 2,3'''
import joblib 
from sklearn.impute import KNNImputer

imputer_cache_path = "knn_imputed2.pkl"

try:
    X_train_imputed, test_imputed = joblib.load(imputer_cache_path)
    print("Loaded cached KNN imputed data.")

    ########################################################################
    # X_train_imputed = X_train_imputed.drop(columns=out_list)
    # test_imputed = test_imputed.drop(columns=out_list)
    ########################################################################
except FileNotFoundError:
    numeric_filled_columns = numeric_null_ratio[numeric_null_ratio < 0.8].index  # 결측치 비율 80% 미만 컬럼 선택
    
    knn_imputer = KNNImputer(n_neighbors=6)
    
    X_train_imputed = X_train.copy()
    test_imputed = test.copy()
    
    X_train_imputed[numeric_filled_columns] = knn_imputer.fit_transform(X_train[numeric_filled_columns])
    print("X_train finish")
    test_imputed[numeric_filled_columns] = knn_imputer.transform(test[numeric_filled_columns])
    print("All columns imputed.")
    
    joblib.dump((X_train_imputed, test_imputed), imputer_cache_path)
    print("Saved KNN imputed data.")

In [ ]:
X_train_imputed['임신 시도 또는 마지막 임신 경과 연수'] = X_train_imputed['임신 시도 또는 마지막 임신 경과 연수'].fillna(0)
test_imputed['임신 시도 또는 마지막 임신 경과 연수'] = test_imputed['임신 시도 또는 마지막 임신 경과 연수'].fillna(0)

--Numeric Impute only for Model 1

##### -Categorical

In [ ]:
''' Categorical Drop'''
# ordinal / non ordinal categorical columns 구분
''' ordinal 의 경우 결측치 무의미 '''
''' non ordinal categorical drop '''
# 순서가 없는 categorical columns 중 결측치 90% 이상인 columns drop

categorical_null_columns = []
print(X_train_imputed[Categorical_non_ordinal].isnull().sum())
categorical_null_columns = [ col for col in categorical_columns if X_train_imputed[col].isnull().sum() != 0 ]

# print(categorical_null_columns)

categorical_null_ratio = X_train_imputed[categorical_null_columns].isnull().mean().sort_values(ascending=False)
print(categorical_null_ratio)

drop_columns = categorical_null_ratio[categorical_null_ratio >= 0.9].index
X_train_imputed = X_train_imputed.drop(columns=drop_columns)
test_imputed = test_imputed.drop(columns=drop_columns)

categorical_columns = list(set(categorical_columns) - set(drop_columns))
print(f"남은 categorical_columns: {categorical_columns}")
print(len(categorical_columns))
print("📌"+drop_columns)

--Categorical Impute only for Model 2,3

In [ ]:
''' Categorical Impute2 '''
#model3
imputer3 = SimpleImputer(strategy='most_frequent')

X_train_imputed3 = X_train_imputed.copy()
test_imputed3 = test_imputed.copy()

X_train_imputed3[categorical_columns] = imputer3.fit_transform(X_train_imputed[categorical_columns])
test_imputed3[categorical_columns] = imputer3.transform(test_imputed[categorical_columns])

### 이상치

In [ ]:
# # 이상치 비율 계산
# def calculate_outlier_ratio(data):
#     # 각 컬럼에 대해 이상치를 판단하는 함수
#     Q1 = np.percentile(data, 25)
#     Q3 = np.percentile(data, 75)
#     IQR = Q3 - Q1
#     lower_bound = Q1 - 1.5 * IQR
#     upper_bound = Q3 + 1.5 * IQR
    
#     # 이상치 비율 계산
#     outliers = ((data < lower_bound) | (data > upper_bound)).sum()
#     total = len(data)
#     return outliers / total

# # 이상치 비율 계산
# outlier_ratios = {}
# for column in numeric_columns:
#     outlier_ratios[column] = calculate_outlier_ratio(X_train_imputed[column])

# # 이상치 비율 출력
# for column, ratio in outlier_ratios.items():
#     print(f"{column}의 이상치 비율: {ratio:.2f}")


In [ ]:
# from sklearn.neighbors import LocalOutlierFactor


# print(f"LOF 적용 전 데이터 크기: {X_train_imputed3.shape}")

# lof = LocalOutlierFactor(n_neighbors=30, contamination=0.07)
# outliers = lof.fit_predict(X_train_imputed3[numeric_columns])

# X_train_imputed3 = X_train_imputed3[outliers == 1]
# y = y[outliers == 1]

# print(f"LOF 적용 후 데이터 크기: {X_train_imputed3.shape}")

## 3-3 Scaler

In [ ]:
scaler = MinMaxScaler()
# scaler = StandardScaler()

X_train_scaled = X_train_imputed3.copy()
test_scaled = test_imputed3.copy()

X_train_scaled[numeric_columns] = scaler.fit_transform(X_train_imputed3[numeric_columns])
test_scaled[numeric_columns] = scaler.transform(test_imputed3[numeric_columns])
print(X_train_imputed3.shape)


# 4. PreProcessing2

In [ ]:
X_train_numeric = X_train_imputed3[numeric_columns]
test_numeric = test_imputed3[numeric_columns]

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_numeric)
test_scaled = scaler.transform(test_numeric)

# PCA 적용 (설정: 분산을 95% 유지하는 주성분 개수 자동 선택)
pca = PCA(n_components=0.95, random_state=42)
X_train_pca = pca.fit_transform(X_train_scaled)
test_pca = pca.transform(test_scaled)

# PCA 결과를 데이터프레임으로 변환
X_train_pca = pd.DataFrame(X_train_pca, columns=[f'PC{i+1}' for i in range(X_train_pca.shape[1])])
test_pca = pd.DataFrame(test_pca, columns=[f'PC{i+1}' for i in range(test_pca.shape[1])])

# PCA 설명된 분산 비율 확인
explained_variance = pca.explained_variance_ratio_
cumulative_variance = explained_variance.cumsum()
num_components = len(cumulative_variance)

print(f"PCA :  사용된 주성분 개수: {num_components}")
print(f"설명된 분산 비율 (누적): {cumulative_variance[-1]:.4f}")

# 기존 데이터에서 숫자형 부분을 PCA 결과로 교체
X_train_imputed3.update(X_train_pca)
test_imputed3.update(test_pca)


### 4-2. 세제곱 변환

In [ ]:
for col in list(set(numeric_columns)-set(['이식된 배아 수'])):
  X_train_imputed3[col] = X_train_imputed3[col] ** 2
  test_imputed3[col] = test_imputed3[col] ** 2

### 4-3. Feature 생성

#### 4-3-1. 단순 생성성

In [ ]:
X_train_imputed3['시술 시점'] = X_train_imputed3['시술 시기 코드'] + '_' + X_train_imputed3['시술 당시 나이']
test_imputed3['시술 시점'] = test_imputed3['시술 시기 코드'] + '_' + test_imputed3['시술 당시 나이']
categorical_columns.append('시술 시점')
Categorical_non_ordinal.append('시술 시점')

# X_train_imputed3['배아 정보'] = (X_train_imputed3['총 생성 배아 수'] * X_train_imputed3['해동된 배아 수'])
# numeric_columns.append('배아 정보')

# print(X_train_imputed['배아 정보']



#### 4-3-2. 결측치 보완성

In [ ]:
X_train_newf = X_train_imputed3.copy()
test_newf = test_imputed3.copy()
y_train3 = y

# 5. Model

#### model

In [ ]:
''' train / valid 구분 '''  #70% train
#model3
X_train3 = X_train_newf[:int(X_train.shape[0]*0.7):]
X_valid3 = X_train_newf[int(X_train.shape[0]*0.7):]
y_train3 = y[:int(X_train.shape[0]*0.7):]
y_valid3 = y[int(X_train.shape[0]*0.7):]

In [ ]:
''' model 3 '''

#categorical str transform
for col in categorical_columns:
    # print(type(X_train[col]))
    
    X_train3[col] = X_train3[col].astype(str)
    X_valid3[col] = X_valid3[col].astype(str)
    # test[col] = test[col].astype(str)

# print(X_train3[categorical_columns].dtypes)

#pool
''' categorical 내장 encoding '''
train_pool = Pool(data = X_train3, label = y_train3, cat_features = categorical_columns)
val_pool = Pool(data = X_valid3, label = y_valid3, cat_features = categorical_columns)

# classes = np.unique(y_train3)
# weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train3)
# class_weights = dict(zip(classes, weights))

#fitting
model3 = CatBoostClassifier(cat_features = categorical_columns, random_state = 42, silent=True, eval_metric='AUC')
model3.fit(train_pool, eval_set=val_pool)

y_pred_prob = model3.predict_proba(X_valid3)[:, 1]
auc_score = roc_auc_score(y_valid3, y_pred_prob)
print(f"Validation AUC-ROC: {auc_score:.8f}")

In [ ]:
''' model 3 '''

from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_auc_score

# categorical 변수 문자열 변환
for col in categorical_columns:
    X_train3[col] = X_train3[col].astype(str)
    X_valid3[col] = X_valid3[col].astype(str)

# CatBoost Pool 생성
train_pool = Pool(data=X_train3, label=y_train3, cat_features=categorical_columns)
val_pool = Pool(data=X_valid3, label=y_valid3, cat_features=categorical_columns)

# 최적의 하이퍼파라미터 적용
best_params = {
    'n_estimators': 1600,
    'learning_rate': 0.022269562365768235,
    'depth': 6,
    'subsample': 0.8570762892949296,
    'l2_leaf_reg': 2.736528145798781,
    'min_data_in_leaf': 50,
    'colsample_bylevel': 0.7609486820290322,
    'bootstrap_type': 'Bernoulli',
    'early_stopping_rounds': 70,
    'eval_metric': 'AUC',
    'random_state': 42,
    'verbose': 0
}

# 모델 학습
model3 = CatBoostClassifier(cat_features=categorical_columns, **best_params)
model3.fit(train_pool, eval_set=val_pool)

# 예측 및 평가
y_pred_prob = model3.predict_proba(X_valid3)[:, 1]
auc_score = roc_auc_score(y_valid3, y_pred_prob)
print(f"Validation AUC-ROC: {auc_score:.8f}")


In [ ]:
for col in categorical_columns:
    X_train3[col] = X_train3[col].astype(str)
    X_valid3[col] = X_valid3[col].astype(str)

#pool
''' categorical 내장 encoding '''
train_pool = Pool(data = X_train3, label = y_train3, cat_features = categorical_columns)
val_pool = Pool(data = X_valid3, label = y_valid3, cat_features = categorical_columns)

classes = np.unique(y_train3)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train3)
class_weights = dict(zip(classes, weights))

# Objective 함수
def objective(trial):
    params = {
        # 트리 개수: 학습 속도와 성능의 균형을 위해 적당한 범위
        "n_estimators": trial.suggest_int("n_estimators", 800, 2000, step=200),

        # 학습률: 안정성과 속도의 균형을 위해 작은 범위
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.2),

        # 트리 깊이: 가장 효과적인 깊이 범위
        "depth": trial.suggest_int("depth", 5, 10),

        # 부분 샘플링 비율: 과적합 방지를 위해 일반적인 범위
        "subsample": trial.suggest_uniform("subsample", 0.6, 0.9),

        # 정규화: 적은 값과 큰 값 모두 탐색 가능하게 설정
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 2, 10),

        # 리프당 최소 샘플 수: 적당한 범위로 설정
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 5, 50, step=5),

        # 트리 수준별 샘플링 비율: 일반적인 범위
        "colsample_bylevel": trial.suggest_uniform("colsample_bylevel", 0.7, 1.0),

        # 부트스트랩 방법: 가장 자주 사용되는 옵션만 제공
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ['Bayesian', 'Bernoulli', 'MVS']),

        # 클래스 불균형 처리
        "class_weights": class_weights,

        # 범주형 변수 지정
        "cat_features": categorical_columns,

        # 평가 지표
        "eval_metric": "AUC",

        # 조기 중단
        "early_stopping_rounds": trial.suggest_int("early_stopping_rounds", 30, 70, step=10),

        # 로그 최소화
        "verbose": 0
    }

    # Bayesian 방식에서는 subsample 제거
    if params["bootstrap_type"] == "Bayesian":
        del params["subsample"]

    cat_model = CatBoostClassifier(**params)
    cat_model.fit(train_pool, eval_set=val_pool)
    val_pred = cat_model.predict_proba(X_valid3)[:, 1]

    return roc_auc_score(y_valid3, val_pred)

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

best_params = study.best_params
print("Best params:", best_params)

In [ ]:
from sklearn.model_selection import StratifiedKFold

for col in categorical_columns:
    X_train_newf[col] = X_train_newf[col].astype(str)


# KFold 설정
N_SPLITS = 5  # 5-Fold Cross Validation
kf = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

# 클래스 가중치 계산
classes = np.unique(y_train3)
weights = compute_class_weight(class_weight='balanced', classes=classes, y=y_train3)
class_weights = dict(zip(classes, weights))

# Optuna Objective 함수 정의
def objective(trial):
    params = {
        # 트리 개수: 학습 속도와 성능의 균형을 위해 적당한 범위
        "n_estimators": trial.suggest_int("n_estimators", 800, 2000, step=200),

        # 학습률: 안정성과 속도의 균형을 위해 작은 범위
        "learning_rate": trial.suggest_loguniform("learning_rate", 0.01, 0.2),

        # 트리 깊이: 가장 효과적인 깊이 범위
        "depth": trial.suggest_int("depth", 5, 10),

        # 부분 샘플링 비율: 과적합 방지를 위해 일반적인 범위
        "subsample": trial.suggest_uniform("subsample", 0.6, 0.9),

        # 정규화: 적은 값과 큰 값 모두 탐색 가능하게 설정
        "l2_leaf_reg": trial.suggest_loguniform("l2_leaf_reg", 2, 10),

        # 리프당 최소 샘플 수: 적당한 범위로 설정
        "min_data_in_leaf": trial.suggest_int("min_data_in_leaf", 5, 50, step=5),

        # 트리 수준별 샘플링 비율: 일반적인 범위
        "colsample_bylevel": trial.suggest_uniform("colsample_bylevel", 0.7, 1.0),

        # 부트스트랩 방법: 가장 자주 사용되는 옵션만 제공
        "bootstrap_type": trial.suggest_categorical("bootstrap_type", ['Bayesian', 'Bernoulli', 'MVS']),

        # 클래스 불균형 처리
        "class_weights": class_weights,

        # 범주형 변수 지정
        "cat_features": categorical_columns,

        # 평가 지표
        "eval_metric": "AUC",

        # 조기 중단
        "early_stopping_rounds": trial.suggest_int("early_stopping_rounds", 30, 70, step=10),

        # 로그 최소화
        "verbose": 0
    }

    # Bayesian 방식에서는 subsample 제거
    if params["bootstrap_type"] == "Bayesian":
        del params["subsample"]

    # K-Fold Cross Validation 수행
    auc_scores = []

    for train_idx, valid_idx in kf.split(X_train_newf, y_train3):
        X_tr, X_val = X_train_newf.iloc[train_idx], X_train_newf.iloc[valid_idx]
        y_tr, y_val = y_train3.iloc[train_idx], y_train3.iloc[valid_idx]

        train_pool = Pool(data=X_tr, label=y_tr, cat_features=categorical_columns)
        val_pool = Pool(data=X_val, label=y_val, cat_features=categorical_columns)

        # 모델 학습
        cat_model = CatBoostClassifier(**params)
        cat_model.fit(train_pool, eval_set=val_pool)

        # 검증 데이터 예측
        val_pred = cat_model.predict_proba(X_val)[:, 1]

        # AUC 점수 저장
        auc_scores.append(roc_auc_score(y_val, val_pred))

    # 평균 AUC 반환
    return np.mean(auc_scores)

# Optuna Study 생성 및 최적화 실행
study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=50)

# 최적의 하이퍼파라미터 출력
best_params = study.best_params
print("Best parameters:", best_params)


# 6. Submission

In [ ]:
''' model 3 '''
#categorical str transform
for col in categorical_columns:
    X_train_newf[col] = X_train_newf[col].astype(str)
    test_newf[col] = test_newf[col].astype(str)

#pool
''' categorical 내장 encoding '''
train_pool = Pool(data = X_train_newf, label = y, cat_features = categorical_columns)

# classes = np.unique(y)
# weights = compute_class_weight(class_weight='balanced', classes=classes, y=y)
# class_weights = dict(zip(classes, weights))

#fitting
model = CatBoostClassifier(cat_features = categorical_columns, random_state = 42, silent=True, eval_metric='AUC')
model.fit(train_pool)

pred_proba = model.predict_proba(test_newf)[:, 1]

submission = pd.read_csv('./sample_submission.csv')
submission['probability'] = pred_proba

submission.to_csv('./preprocessing_v4-1.csv', index=False)

In [ ]:
''' model 3 '''
# categorical 변수 문자열 변환
for col in categorical_columns:
    X_train_newf[col] = X_train_newf[col].astype(str)
    test_newf[col] = test_newf[col].astype(str)

# CatBoost Pool 생성
train_pool = Pool(data=X_train_newf, label=y, cat_features=categorical_columns)

# 최적의 하이퍼파라미터 적용
# best_params = {
#     'n_estimators': 800,
#     'learning_rate': 0.041481707162322246,
#     'depth': 6,
#     'l2_leaf_reg': 5.192930349370407,
#     'min_data_in_leaf': 10,
#     'colsample_bylevel': 0.8910668655874581,
#     'bootstrap_type': 'Bayesian',
#     'early_stopping_rounds': 60,
#     'eval_metric': 'AUC',
#     'random_state': 42,
#     'verbose': 0
# }

# 모델 학습
model = CatBoostClassifier(cat_features=categorical_columns,l2_leaf_reg=5)
model.fit(train_pool)

# 예측
pred_proba = model.predict_proba(test_newf)[:, 1]

# 제출 파일 생성
submission = pd.read_csv('./sample_submission.csv')
submission['probability'] = pred_proba

submission.to_csv('./preprocessing_v9.csv', index=False)


In [ ]:
print(pred_proba)